In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
import datetime
from sqlalchemy import create_engine
import geopandas as gpd
import geoplot as gplt
import contextily as ctx
from shapely.geometry import Point, Polygon
import descartes
%matplotlib inline

/Users/ju/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## Organizing data to be used in Tableau

In [2]:
engine = create_engine("sqlite:///stations.db")

In [5]:
geo_data = pd.read_sql("SELECT STATION,LAT,LONG FROM stations_table WHERE STATION IN ('34 St-Penn Station','Grand Central-42 St','23 St','34 St-Herald Sq','Times Sq-42 St','42 St-Port Authority Bus Terminal','14 St-Union Sq','Fulton St','86 St','Canal St','125 St','') GROUP BY STATION;", engine)

In [6]:
geo_data

,STATION,LAT,LONG
0,14 St-Union Sq,40.735736,-73.990568
1,23 St,40.741303,-73.989344
2,34 St-Herald Sq,40.749567,-73.987950
3,34 St-Penn Station,40.752287,-73.993391
4,42 St-Port Authority Bus Terminal,40.757308,-73.989735
5,86 St,40.622687,-74.028398
6,Canal St,40.719527,-74.001775
7,Fulton St,40.710374,-74.007582
8,Grand Central-42 St,40.751776,-73.976848
9,Times Sq-42 St,40.754672,-73.986754


In [8]:
geo_data.to_csv('topstations_geo_updated.csv',header=False,index=False)

In [ ]:
geo_data_top20 = pd.read_sql("SELECT STATION,LAT,LONG FROM stations_table WHERE STATION IN ('42 St-Port Authority Bus Terminal','23 St','34 St-Herald Sq','Canal St','125 St','Times Sq-42 St','Chambers St','34 St-Penn Station','Elmhurst Av','14 St') GROUP BY STATION;", engine)

In [9]:
engine2 = create_engine("sqlite:///mta_sample.db")

In [21]:
covid = pd.read_sql("SELECT BOROUGH FROM zipcode_data WHERE BOROUGH LIKE 'B%';",engine2)
covid

,BOROUGH
0,Bronx
1,Bronx
2,Bronx
3,Bronx
4,Bronx
...,...
57,Brooklyn
58,Brooklyn
59,Brooklyn
60,Brooklyn


In [26]:
bk = pd.read_sql("SELECT BOROUGH, SUM(COVID_CASES) AS TOTAL FROM zipcode_data WHERE BOROUGH = 'Brooklyn' ORDER BY TOTAL DESC;",engine2)

In [27]:
bk

,BOROUGH,TOTAL
0,Brooklyn,216436


In [28]:
mn = pd.read_sql("SELECT BOROUGH, SUM(COVID_CASES) AS TOTAL FROM zipcode_data WHERE BOROUGH = 'Manhattan' ORDER BY TOTAL DESC;",engine2)

In [29]:
mn

,BOROUGH,TOTAL
0,Manhattan,101204


In [31]:
totals = pd.read_sql("SELECT BOROUGH, SUM(COVID_CASES) AS TOTAL FROM zipcode_data GROUP BY BOROUGH ORDER BY TOTAL DESC;",engine2)

In [32]:
totals

,BOROUGH,TOTAL
0,Brooklyn,216436
1,Queens,214038
2,Bronx,142273
3,Manhattan,101204
4,Staten Island,59440


In [33]:
avg = pd.read_sql("SELECT BOROUGH, AVG(COVID_CASES) AS AVERAGE FROM zipcode_data GROUP BY BOROUGH ORDER BY AVERAGE DESC;",engine2)
avg

,BOROUGH,AVERAGE
0,Brooklyn,5849.621622
1,Bronx,5690.920000
2,Staten Island,4953.333333
3,Queens,3627.762712
4,Manhattan,2300.090909


In [15]:
covid_data = pd.read_sql('SELECT ZIP, NEIGHBORHOOD, LAT, LONG, COVID_CASES, COVID_DEATHS FROM zipcode_data ORDER BY COVID_CASES DESC limit 20;',engine2)

In [16]:
covid_data

,ZIP,NEIGHBORHOOD,LAT,LONG,COVID_CASES,COVID_DEATHS
0,11368,Corona/North Corona,40.749594,-73.852604,11672,521
1,11230,Midwood,40.622164,-73.965096,11108,382
2,10314,Bloomfield/Freshkills Park,40.599333,-74.166462,10935,337
3,10467,Allerton/Norwood/Pelham Parkway/Williamsbridge,40.880939,-73.873956,10501,442
4,11208,Cypress Hills/East New York,40.668385,-73.871084,10190,356
5,11235,Brighton Beach/Manhattan Beach/Sheepshead Bay,40.583915,-73.949669,9901,517
6,11385,Glendale/Ridgewood,40.701058,-73.890159,9775,211
7,11373,Elmhurst,40.738823,-73.878500,9712,360
8,11223,Gravesend/Homecrest,40.597169,-73.973422,9346,230
9,11220,Sunset Park,40.641081,-74.016495,9017,271


In [17]:
covid_data.to_csv('covid_top_ten_zips.csv',header=False,index=False)